In [ ]:
%matplotlib inline
import os, re, fnmatch, sys
import numpy as np
import pandas as pd
from pathlib import Path
import time

# define the current path (notebooks in lab_utils)
currpath = os.getcwd()
labutilspath = str(Path(currpath).parents[1])
sys.path.append(labutilspath)

from _helpers.basics import rock_info
from ner_usa import tools as usa


# define paths
basepath = '/media/damiansa/data/lab/data/usa_experiments'

# set the asdatapath accordingly (where is the atuoscan data inside basedatapath?)
datafolder   = 'exports'

# construct datapath
datapath = os.path.join(basepath, datafolder)

In [ ]:
# load rock information
ri = rock_info()
io = usa.io(basepath = basepath, datadir = datafolder)

In [ ]:
io.plot_ports('tc')

In [ ]:
io.plot_ports('src')

In [ ]:
import sqlite3
import io as pyio
import json, yaml, ast

def convert_buffer(buffer, round=True, decimals = 2):
    out = np.load(pyio.BytesIO(buffer))['v']
    if round:
        out = np.around(out, decimals = decimals)
    return out

def fix_infovals(info):
    h = info.replace('u','')
    d = yaml.load(h, yaml.SafeLoader)
    d = pd.Series(d)
    return d

infocols = ['project','sample', 'poreflids','thermocople_location_set','thermocople_locations_list']
# io.dbpath
con = io.db_connect()
# c = con.cursor()


In [ ]:
# c = con.cursor()
# c.execute("SELECT mnem,data FROM database_trace")
# mnem, buffer = c.fetchone()
# data = convert_buffer(buffer)
# #data = np.load(StringIO(b))['v']
# #print(data)
# # fetch run data
# # reader = con.execute("SELECT id, infovals, extra FROM database_run")
# # # run = reader.fetchall()

In [ ]:

# c = con.cursor()
# c.row_factory = sqlite3.Row
# c.execute("SELECT id, infovals FROM database_run")
# # # c.fetchall()
# run = [dict(row) for row in c.fetchall()]
# c.close()

In [ ]:


#json.loads(h.replace("'", "\""))
#type(h)#.replace("'",'"'))
#pd.DataFrame(pd.DataFrame(run).iloc[:,1].values.tolist())
# rows = con.execute("SELECT name FROM sqlite_master WHERE type = 'table'")
# tables = [row[0] for row in rows]

In [ ]:


c = con.cursor()
# c.row_factory = sqlite3.Row
c.execute("SELECT id, infovals FROM database_run")
data = c.fetchall()
df = pd.DataFrame(data, columns = ['id', 'infovals'])#.apply(fix_infovals)
df.dropna(inplace=True)
df.reset_index(inplace=True, drop = True)
df = pd.concat([df.loc[:,'id'], df.loc[:,'infovals'].apply(fix_infovals).loc[:,infocols]], axis = 1)
df.replace(to_replace='None', value=np.nan, inplace = True)
df.dropna(inplace=True, subset=['thermocople_locations_list'])
df.reset_index(inplace=True, drop = True)
df.head()

In [ ]:
st = time.time()
c = con.cursor()
# c.row_factory = sqlite3.Row
# c.execute("SELECT abbrev,format,data FROM database_trace WHERE capture_id=109")
c.execute("SELECT abbrev,data FROM database_trace WHERE capture_id=109")
data = c.fetchall()
c.close()
df = pd.DataFrame(data, columns = ['tc','data']).iloc[:-1,:]
trace = df.loc[:,'data'].apply(convert_buffer).apply(pd.Series)#.apply(np.around, decimals=2)
df = pd.concat([df.loc[:,'tc'], trace], axis=1)
print(time.time() - st)
df.head()

In [ ]:


# d = df.loc[:,'data'].values
# d = np.concatenate(d)
# d = np.reshape(d, (df.shape[0], int(d.shape[0]/df.shape[0])))
# d.shape
# st = time.time()
# c = con.cursor()
# # c.row_factory = sqlite3.Row
# # c.execute("SELECT abbrev,format,data FROM database_trace WHERE capture_id=109")
# c.execute("SELECT abbrev,data FROM database_trace WHERE capture_id=109")
# data = c.fetchall()
# ds = []
# for row in data:
#     d = convert_buffer(row[1])
#     ds.append(d)
# df2 = pd.DataFrame(ds)
# print(time.time() - st)

In [ ]:
def sql_identifier(s):
    return '"' + s.replace('"', '""') + '"'

def get_col_names(cursor, tablename):
    """Get column names of a table, given its name and a cursor
       (or connection) to the database.
    """
    reader=cursor.execute("SELECT * FROM {}".format(tablename))
    return [x[0] for x in reader.description] 


for table in tables:
    print("\n\ntable: " + table)
    cols = get_col_names(cur, table)
    print(cols)
    # print('table info', sep = '\n')
    # rows = con.execute("PRAGMA table_info({})".format(sql_identifier(table)))
    # print(rows.fetchall())
    print('\nforeign keys')
    rows = con.execute("PRAGMA foreign_key_list({})".format(sql_identifier(table)))
    print(rows.fetchall())